In [1]:
from pathlib import Path

from datetime import datetime

from ragdaemon.daemon import Daemon

annotators = {
    "hierarchy": {},
    # "chunker": {},
    # "call_graph": {},
    # "summarizer": {},
    # "diff": {},
}

sample_dir = Path.home() / "django"
# sample_dir = Path.cwd()
start_time = datetime.now()
daemon = Daemon(
    cwd=sample_dir, 
    annotators=annotators, 
    verbose=2
)
await daemon.update()
print("Elapsed time:", datetime.now() - start_time)

# Baseline: 1:12, 1:11
# Remove bm25: 01:09, 1:11
# lru_cache is_text_encoded: 1.05, 1.06

Initialized empty graph.
Initializing annotators: ['hierarchy']...
Saved updated graph to /Users/granthawkins/.mentat/ragdaemon/ragdaemon-django.json
Elapsed time: 0:00:04.696033


In [2]:
import json

with open('timestamps.json') as f:
    timestamps = json.load(f)
# Sort alphabetically
timestamps = dict(sorted(timestamps.items()))
last_time = None
for name, time in timestamps.items():
    if last_time is not None:
        print(name, time - last_time)
    last_time = time

02-got_paths 0.6188850402832031
03-load_files 0.30980491638183594
04-load_dirs 65.60865926742554
05-sync_db 0.09315681457519531


In [3]:
directory_nodes = [
    node for node, data in daemon.graph.nodes(data=True)
    if data["type"] == "directory"
]
print(len(directory_nodes))

3221


In [3]:
from pathlib import Path
from ragdaemon.daemon import Daemon

cwd = Path.home() / "Desktop" / "testrepo"
daemon = Daemon(cwd)
await daemon.update()
print([n for n in daemon.graph.nodes])

['file1.py', 'ROOT', 'file1.py:BASE', 'file1.py:hello', 'DEFAULT']


In [4]:
# Create a new file, file2, with hello world
file2 = cwd / "file2"
file2.write_text("hello world")
# Stage it with git
# import subprocess
# subprocess.run(["git", "add", "file2"], cwd=cwd)
# subprocess.run(["git", "commit", "-m", "add file2"], cwd=cwd)

await daemon.update()
print([n for n in daemon.graph.nodes])


['file2', 'file1.py', 'ROOT', 'file1.py:BASE', 'file1.py:hello', 'DEFAULT']


In [1]:
from ragdaemon.get_paths import get_paths_for_directory
from ragdaemon.annotators.hierarchy import files_checksum


daemon.pipeline['hierarchy'].is_complete(daemon.graph, None)
print(daemon.graph.graph.get("files_checksum"))
print(files_checksum(cwd))

for path in get_paths_for_directory(cwd):
    print(str((cwd / path).stat().st_mtime))

NameError: name 'daemon' is not defined